In [1]:
import json
import numpy as np
import random
import collections
import time
import tensorflow as tf
from tensorflow.contrib import rnn

C:\Users\balienig\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
start_time = time.time()

def elapsed(sec):
    if sec<60:
        return str(sec) + " sec"
    elif sec<(60*60):
        return str(sec/60) + " min"
    else:
        return str(sec/(60*60)) + " hr"

#logs_path = './rnn_words'
#writer = tf.summary.FileWriter(logs_path)

In [3]:
print("vec loading...")
reverse_dictionary = json.load(open("word2vec/reverse_dictionary.txt"))
dictionary = {v: int(k) for k, v in reverse_dictionary.items()}
final_embeddings = np.loadtxt('word2vec/final_embeddings.txt')
print("loaded")

vec loading...


FileNotFoundError: [Errno 2] No such file or directory: 'word2vec/reverse_dictionary.txt'

In [4]:
n_input = 3
training_iters = 500000
n_hidden = 512
learning_rate = 0.001

In [5]:
def read_data(file_name):
    '''
     open and read text file
    '''
    text = open(file_name, 'r').readline()
    text = text.split(" ")
    return text

In [6]:
def build_dataset(words):
    count = collections.Counter(words).most_common()
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary

In [7]:
def word_vector(word):
    value = final_embeddings[dictionary[word]]
    return value.tolist() 

In [8]:
def embedding(input_words,output_word,len_unique_words):
    
    dic, re_dic = build_dataset(text)
    
    train_data = []
    target_data = np.zeros((len(input_words), len_unique_words), dtype=float)
    
    for i in range(0,len(input_words)):
        vector = []
        for j in range(0,len(input_words[i])):
            vec = word_vector(input_words[i][j])
            vector.append(vec)
        train_data.append(vector)

    for i in range(0, len(output_word)):
        index = dic[output_word[i]]
        target_data[i][index] = 1.0
    
    return np.array(train_data), target_data

In [9]:
def featurize(text, training_iters):

    unique_words = list(set(text))
    len_unique_words = len(unique_words)
    
    input_words = []
    output_word = []
    
    offset = random.randint(0,n_input+1)
    end_offset = n_input + 1
    
    for i in range(0,training_iters):
        
        if offset > (len(text)-end_offset):
            offset = random.randint(0, n_input+1)
        
        input_words.append(text[offset:offset+n_input])
        output_word.append(text[offset+n_input])
        
        offset += 1
        
    return input_words, output_word, unique_words, len_unique_words

In [10]:
text = read_data("C:\\Users\\balienig\\Documents\\PJ-cPeople-master\\train\\trainRnn\\data\\quotes_.txt")
a, b, unique_words, len_unique_words = featurize(text,700000)
#print(len(a))
#print(input_words[:100])
#print(output_words[:100])
#train_data, target_data = embedding(input_words, output_words, len_unique_words)
print(np.shape(a))

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 97: character maps to <undefined>

In [ ]:
input_words = []
output_words = []
for i in range(0,len(a)):
    if a[i][0] != '.' and a[i][1] != '.':
        input_words.append(a[i])
        output_words.append(b[i])

input_words = input_words[:500000]
output_words = output_words[:500000]

In [ ]:
train_data, target_data = embedding(input_words, output_words, len_unique_words)

In [ ]:
def rnn(x, weight, bias):
    '''
     define rnn cell and prediction
    '''

    x = tf.reshape(x, [-1, n_input * 128])
    x = tf.split(x, n_input, 1)
    

    cell = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    outputs, states = tf.contrib.rnn.static_rnn(cell, x, dtype=tf.float32)
    prediction = tf.matmul(outputs[-1], weight) + bias
    return prediction

In [ ]:
dic, re_dic = build_dataset(text)

acc_total = 0
loss_total = 0
display_step = 1000
    
x = tf.placeholder("float", [None, n_input, 128])
y = tf.placeholder("float", [None, len_unique_words])
    
weight = tf.Variable(tf.random_normal([n_hidden, len_unique_words]))
bias = tf.Variable(tf.random_normal([len_unique_words]))

logits = rnn(x, weight, bias)
prediction = tf.nn.softmax(logits)
softmax = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y)
cost = tf.reduce_mean(softmax)
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)
    
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init_op = tf.global_variables_initializer()
saver = tf.train.Saver()

In [ ]:
allresult = []
with tf.Session() as session:
    
    session.run(init_op)

    #writer.add_graph(session.graph)
    
    for i in range(0,len(train_data)):
        
        input_data = np.reshape(train_data[i], [-1, n_input, 128])
        output_data = np.reshape(target_data[i], [1,-1])

        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, prediction] ,feed_dict={x:input_data, y:output_data})
        
        loss_total += loss
        acc_total += acc
        
        
        if (i+1) % display_step == 0 :
            result = []
            strr = str(i+1)
            l = loss_total/display_step
            print("Iter= " + strr + ", Average Loss= " + \
                  "{:.6f}".format(l) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            
            result.append(strr)
            result.append(l)
            
            symbols_out_pred = re_dic[int(tf.argmax(onehot_pred, 1).eval())]
            print("%s - [%s] vs [%s]" % (input_words[i],output_words[i],symbols_out_pred))
            
            acc_total = 0
            loss_total = 0
            
            allresult.append(result)
    
    save_path = saver.save(session, "./save_lstm1/model.ckpt")
    print("Model saved in file: %s" % save_path)
    print("Optimization Finished!")
    print("Elapsed time: ", elapsed(time.time() - start_time))

In [ ]:
'''
datatest = []
datas = open('/Users/orathai/rnn/Deepcut/datatest.txt')
datas = datas.readlines()
datas = [data[:-1].split(" ") for data in datas]
print(datas[:10])'''

In [ ]:
'''
with tf.Session() as sess:
    saver.restore(sess, "./model/model-1.ckpt")
    print("Model restored.")
    
    input_words = []
    words = datas[0]
    for word in words:
        input_words.append(word_vector(word))
    
    input_data = np.reshape(np.array(input_words), [-1, n_input, 128]) # INPUT TO VECTOR
    # PREDICT
    onehot_pred = sess.run(prediction, feed_dict={x: input_data}) # 1 x 1321
    print(onehot_pred)'''